In [11]:
import os 
import json
import pandas as pd 
import traceback

In [12]:
from langchain.chat_models import ChatOpenAI


In [13]:
from dotenv import load_dotenv
load_dotenv()
KEY=os.getenv('OPENAI_API_KEY')

In [14]:
llm=ChatOpenAI(openai_api_key=KEY,model_name='gpt-3.5-turbo',temperature =0.3)


In [ ]:
llm